# 新闻数据采集器(国内板块)
## 吴泓骏 [拂晓工作室](https://github.com/Errrneist/Alchemist)
* 此程序将对指定股票数据进行收集并整理
* 同时创建SFrame友好的CSV文件以及对数据进行清洗

# 参考资料
* [1] [Basics of SFrame](https://apple.github.io/turicreate/docs/api/generated/turicreate.SFrame.html#turicreate.SFrame)
* [2] [Remove Multiple Substring from String](https://stackoverflow.com/questions/31273642/better-way-to-remove-multiple-words-from-a-string)
* [3] [楼老师的Python分析红楼梦](https://zhuanlan.zhihu.com/p/29209681)
* [4] [如何使用pyltp包进行中文分词](https://blog.csdn.net/sinat_26917383/article/details/77067515)

# 导入库

In [2]:
# 导入库
import urllib
import re
import os
import csv
import time
import datetime
import turicreate as tc
from bs4 import BeautifulSoup

/Users/hongjunwu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 准备阶段
* 1. 定义用于搜寻的板块
* 2. 创建一个包含25个链接的搜寻目录
* 3. 创建一个包含20个Class的搜寻目录

In [3]:
# 新闻板块(同时也会加入热门新闻的数据)
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录板块

# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')

# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个Class的目录！')
# print(class_list)  # Debug

成功创建包含 25 个页面链接的目录！
成功创建包含 20 个Class的目录！


# 获得所有文章的链接

In [4]:
# 初始化urllist
urllist = []
year = '2018'  # Separate parameter
counter = 1

# 创造urllist
print('-----------------获取所有文章超链接程序-----------------')
print('分析任务开始！')
for url in page_list:
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    for each_url in soup.find_all('a', href=True):
        if 'http://finance.eastmoney.com/news/' in each_url['href']:
            if year in each_url['href']:
                urllist.append(each_url['href'])   
    print('第' + str(counter) + '页分析完成！现在一共获得了' + str(len(urllist)) + '篇文章的链接！')
    counter += 1

print('全部分析完成！正在查重...')
urllist = list(set(urllist))
print('任务完成！共获得了' + str(len(urllist)) + '篇文章的链接！')
print('---------------------------------------------')

-----------------获取所有文章超链接程序-----------------
分析任务开始！
第1页分析完成！现在一共获得了37篇文章的链接！
第2页分析完成！现在一共获得了74篇文章的链接！
第3页分析完成！现在一共获得了117篇文章的链接！
第4页分析完成！现在一共获得了161篇文章的链接！
第5页分析完成！现在一共获得了198篇文章的链接！
第6页分析完成！现在一共获得了235篇文章的链接！
第7页分析完成！现在一共获得了274篇文章的链接！
第8页分析完成！现在一共获得了311篇文章的链接！
第9页分析完成！现在一共获得了350篇文章的链接！
第10页分析完成！现在一共获得了388篇文章的链接！
第11页分析完成！现在一共获得了428篇文章的链接！
第12页分析完成！现在一共获得了466篇文章的链接！
第13页分析完成！现在一共获得了507篇文章的链接！
第14页分析完成！现在一共获得了545篇文章的链接！
第15页分析完成！现在一共获得了585篇文章的链接！
第16页分析完成！现在一共获得了622篇文章的链接！
第17页分析完成！现在一共获得了666篇文章的链接！
第18页分析完成！现在一共获得了705篇文章的链接！
第19页分析完成！现在一共获得了744篇文章的链接！
第20页分析完成！现在一共获得了781篇文章的链接！
第21页分析完成！现在一共获得了819篇文章的链接！
第22页分析完成！现在一共获得了856篇文章的链接！
第23页分析完成！现在一共获得了894篇文章的链接！
第24页分析完成！现在一共获得了934篇文章的链接！
第25页分析完成！现在一共获得了975篇文章的链接！
全部分析完成！正在查重...
任务完成！共获得了517篇文章的链接！
---------------------------------------------


# 将各个网页的资料抓取进SFrame
* 1. 新建用于抓取的函数
* 2. 实施抓取
* 3. 清洗数据

# 创建洗词库

In [5]:
# 清洗数据的词库
banned_info = ['责任编辑','原标题']
banned_words = ['摘要\n', '\n', '\r','\u3000','（中国新闻网）',
                '来源','以下简称', '（新华社）', '>>>', 
                '商务微新闻带你一图了解！','-', '经济日报', '中国经济网',
                '附件：', '>>>>', '>>', '>>>>>', '（新华网）', 
                '（第一财经）', '据新华社报道，', '▼', '▲', '【','】',' ']

# 创建抓取与清洗函数

In [24]:
# 抓取新闻URL函数
def collectNews(news, url, counter):
    # 准备请求数据
    req = urllib.request.Request(url)
    print('请求链接连接成功!')
    response = urllib.request.urlopen(req)
    print('收到反馈信号！')
    html = response.read()
    print('HTML生成完毕！')
    soup = BeautifulSoup(html, "lxml")
    print('SOUP创建完成！')
    
    # 获得文章的发表时间 
    time = soup.find(class_="time").get_text()
    print('时间获取完毕！')
    
    # 获得文章的标题
    title = soup.find('h1').get_text()
    
    # 鉴于有的文章将标题重复一遍 如此加入洗词库把标题洗掉
    banned_words.append(title)
    
    # 获得文章的内容
    content = soup.find(id="ContentBody").get_text() 
    
    # 清洗文章末尾编辑原文信息    
    for banned_information in banned_info:
        if banned_information in content:
            content = content[0:re.search(banned_information, content).span()[0]-1].strip()
        
    # 清洗文章中冗余文字
    for banned_word in banned_words:
        content = content.replace(banned_word, '')
    print('文章内容获取完成！')
    
    # 将标题从洗词库中去除
    banned_words.remove(title)

    # 获取相关主题
    related_stocks = []
    for each in soup.find_all(class_='keytip'): 
        related_stocks.append(each.get_text())
    
    # 相关主题查重
    related_stocks = list(set(related_stocks))
    print('相关主题单获取完成！')         
    
    # 写入SFrame
    temp_sframe = tc.SFrame({'year':[str(time[0:4])],
                             'month':[str(time[5:7])],
                             'day':[str(time[8:10])],
                             'date':[str(time[0:4]) + str(time[5:7]) + str(time[8:10])],
                             'title':[title],
                             'contents':[content], 
                             'related':[related_stocks]})
    news = news.append(temp_sframe)
    print('SFrame写入完成!')
    
    # 释放内存
    del(req, response, html, soup, time, content, related_stocks)
    
    # 刷新计数 
    counter += 1
    print('计数刷新完成！')
    print('页面数据获取完毕！')
    return news

# 执行抓取与数据清洗函数

In [ ]:
# 初始化计数器
counter = 1

# 获取总任务数
total = len(urllist)

# 初始化SFrame
news = tc.SFrame({'year':['0000'],'month':['00'],'day':['00'],'date':['00000000'],'title':['Null Title'],'contents':['Null Contents'],'related':[['Null', 'Null']]})

# 下载数据
for each_article in urllist:
    print('=============================================')
    print('正在获取第' + str(counter) + '篇文章，共' + str(total) + '篇.')
    print('---------------------------------------------')
    news = collectNews(news, each_article, counter) 
    counter += 1
    
print('获取完毕！共获取了' + str(len(news['title'])) + '篇文章!')

# 删除占位符
news = news[1:len(news['title'])]

# 若仍然存在 则原地去世
if(news[0]['year'] == '0000'):
    print('未移除前:' + news[0]['title'])
    news = news[1:len(news['title'])]
    print('移除后:' + news[0]['title'])
else:
    print('占位符已确认移除！')

# 最后一次查重
news = news.unique()

正在获取第1篇文章，共517篇.
---------------------------------------------
请求链接连接成功!


# 盖上时间戳并保存数据

In [21]:
# 保存数据
filepath = '../DataSets/Eastmoney/News/China/'
date = '20' + str(datetime.datetime.now().strftime("%y%m%d-%H%M"))
news.save(filepath + 'CHINA' + date + '.csv', format='csv')
print('成功保存数据文件！数据路径：' + filepath + 'CHINA' + '-' + date + '.csv')

# 打印时间戳
print('程序运行时间戳：20' 
      + str(datetime.datetime.now().strftime("%y")) + '年'
      + str(datetime.datetime.now().strftime("%m")) + '月' 
      + str(datetime.datetime.now().strftime("%d")) + '日' 
      + str(datetime.datetime.now().strftime("%H")) + '时' 
      + str(datetime.datetime.now().strftime("%M")) + '分' 
      + str(datetime.datetime.now().strftime("%S")) + '秒')

成功保存数据文件！数据路径：../DataSets/Eastmoney/News/China/CHINA-20180701-2148.csv
程序运行时间戳：2018年07月01日21时48分41秒
